In [1]:
from datasets import load_dataset
import numpy as np
import pandas as pd
from evaluate import load


In [2]:
# data = datasets.load_dataset("DFKI-SLT/cross_ner", "conll2003"
def get_data(dataset, config):
    return load_dataset(dataset, config)



In [3]:
datasets = {
    "universalner/universal_ner": ['ceb_gja', 'zh_gsd', 'zh_gsdsimp', 'zh_pud', 'hr_set', 'da_ddt', 'en_ewt', 'en_pud', 'de_pud', 'pt_bosque', 'pt_pud', 'ru_pud', 'sr_set', 'sk_snk', 'sv_pud', 'sv_talbanken', 'tl_trg', 'tl_ugnayan'],
    "DFKI-SLT/cross_ner" : ['ai', 'conll2003', 'literature', 'music', 'politics', 'science']
}

In [4]:
def label_map(data): # Accessing the label names from the 'ner_tags' feature.
    label_names = data['test'].features['ner_tags'].feature.names
    # Create mapping from label ID to label string name
    get_label = {k: v for k, v in enumerate(label_names)} 
# Create reverse mapping from label name to label ID
    get_id = {v: k for k, v in enumerate(label_names)}
    return lambda x : get_label[x], lambda label : get_id[label]



In [5]:
def recognize_named_entities(dataset):
    id2label, label2id = label_map(dataset)
    border = int(len(dataset['test']) // 2)
    # print(border)
    train_df = pd.DataFrame(dataset['test']).iloc[:border] if len(dataset) == 1 else pd.DataFrame(dataset['train'])
    test_df = pd.DataFrame(dataset['test']).iloc[border:] if len(dataset) == 1 else pd.DataFrame(dataset['test'])
    # validation_df = pd.DataFrame(dataset['validation'])
    # train_df = pd.concat([train_df, validation_df], ignore_index=True)
    named_entity_dict = {(tok, id2label(prev)): id2label(tag) for _, row in train_df.iterrows() for tok, prev, tag in zip(row['tokens'], [0] + row['ner_tags'][:-1], row['ner_tags']) if tag}
    recognized_entities = []
    for sentence in test_df['tokens']:
        last_tag = 'O'
        pred_sentence = []
        for word in sentence:
            if (word, last_tag) in named_entity_dict:
                last_tag = named_entity_dict[(word, last_tag)]
                pred_sentence.append(last_tag)
            else:
                last_tag = 'O'
                pred_sentence.append(last_tag)
        recognized_entities.append(pred_sentence)
    metric = load("seqeval")
    res = metric.compute(predictions=recognized_entities, references=[[id2label(tag) for tag in sentence]  for sentence in test_df['ner_tags']])
    return res['overall_f1'], res['overall_precision'], res['overall_recall'] 

In [6]:
for dataset, configs in datasets.items():
    for cfg in configs:
        print(cfg)
        data = get_data(dataset, cfg)
        print(data['test'].features['ner_tags'].feature.names)
        f1, precision, recall = recognize_named_entities(data)
        print(f"precision: {precision}, recall: {recall}, f1: {f1}\n")




ceb_gja


Using the latest cached version of the module from /home/kirill/.cache/huggingface/modules/datasets_modules/datasets/universalner--universal_ner/305d16cdde14f6b2d7922615d92315c452ed9200a563be4a06839d846eb34cdc (last modified on Fri Jul  5 13:18:19 2024) since it couldn't be found locally at universalner/universal_ner, or remotely on the Hugging Face Hub.


ModuleNotFoundError: No module named 'conllu'